In [3]:
import pandas as pd

orders=pd.read_csv("/content/orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users=pd.read_json("/content/users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [10]:
import sqlite3
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("/content/restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [12]:
merged_1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

merged_1.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [13]:
final_df = pd.merge(
    merged_1,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [16]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [17]:
import pandas as pd

df = pd.read_csv("/content/final_food_delivery_dataset.csv")
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [18]:
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue



,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [19]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [20]:
user_spend = df.groupby("user_id")["total_amount"].sum()
count_users = user_spend[user_spend > 1000].count()

count_users


np.int64(2544)

In [21]:
def rating_range(r):
    if r <= 3.5:
        return "3.0–3.5"
    elif r <= 4.0:
        return "3.6–4.0"
    elif r <= 4.5:
        return "4.1–4.5"
    else:
        return "4.6–5.0"

df["rating_range"] = df["rating"].apply(rating_range)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [22]:
(
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [26]:
cuisine_analysis = df.groupby("cuisine").agg(
    restaurant_count=("restaurant_name_x", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("restaurant_count")

cuisine_analysis


,restaurant_count,total_revenue
cuisine,,
Chinese,115,1930504.65
Indian,122,1971412.58
Italian,124,2024203.80
Mexican,124,2085503.09


In [24]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [27]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    order_count=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
    .sort_values("avg_order_value", ascending=False)


,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


In [28]:
combo_revenue = df.groupby(["membership", "cuisine"])["total_amount"].sum()
combo_revenue.sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [29]:
restaurant_stats.loc[restaurant_stats.index.isin([
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
])].query("order_count < 20").sort_values("avg_order_value", ascending=False)



,order_count,avg_order_value
restaurant_name_x,,
Ruchi Foods Chinese,19,686.603158


In [30]:
# Filter only the 4 required combinations
filtered = df[
    ((df["membership"] == "Gold") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Gold") & (df["cuisine"] == "Italian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Chinese"))
]

# Calculate total revenue for each combination
combo_revenue = (
    filtered
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


,,total_amount
membership,cuisine,
Gold,Italian,1005779.05
Regular,Indian,992100.27
Gold,Indian,979312.31
Regular,Chinese,952790.91


In [31]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-3625254343.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [32]:
gold_orders_count = df[df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [33]:
hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [34]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [35]:
gold_avg_order_value = df[df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)


np.float64(797.15)

In [36]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [38]:
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [39]:
gold_top_city_orders = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_gold_city)
].shape[0]

gold_top_city_orders


1337

In [40]:
df.shape[0]


10000